In [21]:
from Optimization.Algorithms.Algorithm import Algorithm
from Numerical_Objects.Vector import Vector

import numpy as np 

In [66]:
class DolphinSearch(Algorithm):

    def __init__(self, N, T1, T2, speed, M, A, e):
        self._Dol = None
        self._k = None
        self._f_k = None
        self._l = None
        self._TS = np.full((N, N), np.inf)
        self._N = N
        self._T1 = T1
        self._T2 = T2
        self._speed = speed
        self._M = M
        self._A = A
        self._e = e

    @classmethod
    def from_dict(cls, dict_data):
        raise Exception("I don't want to do it and I'll not")

    @classmethod
    def from_json(cls, json_data):
        raise Exception('To be not-json or not to be not-json?')

    def to_dict(self):
        raise Exception('Why?')

    def to_json(self):
        raise Exception('No.')

    @property
    def current_state(self):
        raise Exception("U definitely don't need it!")

    @property
    def iterations(self):
        return [self.get_new_solution,
                # self.information_transfer,
                # self.get_optimal_solution
        ]

    def initialize(self, f, area, seed = None):
        self._Dol = []
        self._f_k = []
        for i in range(self._N):
            point = {k: np.random.uniform(low, up) for k, (low, up) in area.items()}
            point = Vector(point)
            self._Dol.append(point)
            self._f_k.append(f(point))
        self._k = [dol.copy() for dol in self._Dol]
        for f in alg._f_k:
            print(f)

    def get_new_solution(self, f, area):
        self._l = []
        fit_l = []
        area_v = {k: (-1, 1) for k in area.keys()}

        for i in range(self._N):
            V = []
            for m in range(self._M):
                velocity = Vector({k: np.random.uniform(low, up) 
                                   for k, (low, up) in area_v.items()})
                velocity = velocity * (self._speed / velocity.length)
                V.append(velocity)
            for m in range(self._M):
                print('V: ', V[i])

            sol = [self._Dol[i] + v * t for v in V for t in range(1, self._T1 + 1)]
            sol = [s.constrain(area) for s in sol]
            fit = list(map(f, sol))
        
            idx = np.argmin(fit)
            sol_l = sol[idx]
            print('sol_l: ', sol_l)
            self._l.append(sol_l)
            print('fit_l:', fit[idx])
            fit_l.append(fit[idx])
            
            if fit_l[i] < self._f_k[i]:
                self._k[i] = self._l[i].copy()
                self._f_k[i] = fit_l[i]
        # return self.information_transfer

In [67]:
alg = DolphinSearch(N=2, T1=2, T2=3, speed=5, M=2, A=5, e=2)

def mul(x):
    result = 0
    for val in x.values:
        result += val ** 2 
    return result

area = {'x1': (-10, 10), 'x2': (-10, 10)}
alg.initialize(mul, area)
alg.get_new_solution(mul, area)

31.576218709594198
65.55816014499712
V:  (x1: 0.11560430653408232) x (x2: -4.99866338577732)
V:  (x1: 0.11560430653408232) x (x2: -4.99866338577732)
sol_l:  (x1: 4.969692289121039) x (x2: -2.167753891127397)
fit_l: 29.396998381047087
V:  (x1: -4.435481299242383) x (x2: 2.3079223652608216)
V:  (x1: -4.435481299242383) x (x2: 2.3079223652608216)
sol_l:  (x1: 3.0588957766201155) x (x2: 5.37263644487474)
fit_l: 38.22206574102066


In [68]:
for f in alg._f_k:
    print(f)

29.396998381047087
38.22206574102066
